Go over notes and build up from simpler models

1. R AE 
2. R-P AE 
3. R encoder and TS decoder 
4. R-P encoder, TS decoder

In [ ]:
import torch
import torch.nn as nn
from torch_geometric.nn import GCNConv, GAE
from torch_geometric.utils import train_test_split_edges
from ts_vae.data_processors.grambow_processor import ReactionDataset

## Reactant Encoder

In [ ]:
class MolEncoder(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(MolEncoder, self).__init__()
        self.conv = GCNConv(in_channels, out_channels)

    def forward(self, x, edge_index):
        return self.conv(x, edge_index)


In [ ]:
# normal:
base_path = r'data/'
# azure base_path = r'Users/rmhavij/3d-reactions/data/'
r_dataset = ReactionDataset(base_path, geo_file = 'train_r') 

data = r_dataset.data
data.train_mask = data.val_mask = data.test_mask = data.y = None
data = train_test_split_edges(data = data, val_ratio = 0, test_ratio = 0.2)

num_node_fs = r_dataset.data.num_node_features # = 11
out_channels = 2

# build model and optimiser
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GAE(LinearEncoder(num_node_fs, out_channels))
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)
opt = torch.optim.Adam(model.parameters(), lr=0.01)
